<a href="https://colab.research.google.com/github/abhishekmani12/Dreambooth-StableDiffusion/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Dreambooth Setup
# @markdown Change to GPU Runtime. Runtime > Change runtime type > t4 GPU > save

# @markdown Run the cell and restart runtime
import torch
import os
import shutil

!pip install -U autotrain-advanced > logs.txt
!autotrain setup > logs.txt
os.makedirs("/content/Model-Output", exist_ok=True)
os.makedirs("/content/Image-Input", exist_ok=True)

In [ ]:
# @title Training Script
# @markdown Run the cell and then fill it

# @markdown Pre-Prompt: A description of your dataset (Photo of Lewis Hamilton). In this case "Lewis Hamilton" will be used for reference while inferencing.

# @markdown Upload your photos with the upload button and click on submit

# @markdown Takes 8 mins on average for every 100 steps
from IPython.display import display
from PIL import Image
import ipywidgets as widgets
import io

p1 = '/content/Image-Input'


if not os.path.exists(p1) :
    os.makedirs(p1)

else:
    shutil.rmtree(p1)
    os.makedirs(p1)


heading = widgets.HTML("<h2>Training Dreambooth</h2>")

PROMPT = widgets.Text(description='Pre-Prompt:')

BATCH_SIZE = widgets.IntSlider(description='Batch Size:', min=0, max=10, value=1)

STEP_NUMBER = widgets.IntSlider(description='Step Num:', min=100, max=1000, value=500)

upload=widgets.FileUpload(
    accept='image/*',
    multiple=True
)

submit = widgets.Button(description='TRAIN')

def train(PROMPT, BATCH_SIZE, STEP_NUMBER):

  os.environ["prompt"] = PROMPT
  os.environ["batch_size"] = str(BATCH_SIZE)
  os.environ["step_number"] = str(STEP_NUMBER)

  !autotrain dreambooth \
  --model stabilityai/stable-diffusion-xl-base-1.0 \
  --project-name Train-Dreambooth \
  --image-path /content/Image-Input/ \
  --prompt "${prompt}" \
  --resolution 512 \ #can be set to 1024 for better images
  --batch-size ${batch_size} \
  --num-steps ${step_number} \
  --xformers \
  --fp16 \
  --gradient-checkpointing \
  --gradient-accumulation 4\
  --lr 1e-4 \
  --use-8bit-adam

def submit_clicked(b):

    for img_name, file_info in upload.value.items():
      img = Image.open(io.BytesIO(file_info['content']))
      img.save(f"/content/Image-Input/{img_name}")

    train(PROMPT.value, BATCH_SIZE.value, STEP_NUMBER.value)
    print("Training Complete. Model weights is saved in Train-Dreambooth")

submit.on_click(submit_clicked)

display(heading, PROMPT, BATCH_SIZE, STEP_NUMBER, upload, submit)

HTML(value='<h2>Training Dreambooth</h2>')

Text(value='', description='Pre-Prompt:')

IntSlider(value=1, description='Batch Size:', max=10)

IntSlider(value=500, description='Step Num:', max=1000, min=100)

FileUpload(value={}, accept='image/*', description='Upload', multiple=True)

Button(description='TRAIN', style=ButtonStyle())

1
> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='/content/Image-Input/', class_image_path=None, prompt='Photo of Lewis Hamilton', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, project_name='Train-Dreambooth', seed=42, resolution=512, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epochs=1, num_steps=100, checkpointing_steps=100000, resume_from_checkpoint=None, gradient_accumulation=4, gradient_checkpointing=True, lr=0.0001, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=True, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xformers=True, pre_compute_text_embeddings=None, tokenizer_max_length=None, text_encoder_use_att

In [ ]:
# @title Inference Setup
# @markdown Run only once
from IPython.display import display
from PIL import Image
import ipywidgets as widgets
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch
model_path = "/content/Train-Dreambooth"
model = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = DiffusionPipeline.from_pretrained(model, torch_dtype=torch.float16)
pipe.to("cuda")
pipe.load_lora_weights(model_path, weight_name="pytorch_lora_weights.safetensors")

'''
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(

     "stabilityai/stable-diffusion-xl-refiner-1.0",
     torch_dtype=torch.float16,
)

refiner.to("cuda")
'''

In [ ]:
# @title Generate
heading = widgets.HTML("<h2>Inferencing</h2>")
PROMPT_I = widgets.Text(description='Prompt:')
NUM = widgets.BoundedIntText(value=3, min=0, max=20, description="Image Num:")
submit = widgets.Button(description='Generate')

def generate(prompt, num):

  for seed in range(num):
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt, generator=generator, num_inference_steps=25).images[0]
    #image = refiner(prompt=prompt, generator=generator, image=image).images[0]
    image.show()
    image.save(f"Generated-Image-{seed}.png")

def submit_clicked(b):

  generate(PROMPT_I.value, NUM.value)

submit.on_click(submit_clicked)

display(heading, PROMPT_I, NUM, submit)

HTML(value='<h2>Inferencing</h2>')

Text(value='', description='Prompt:')

BoundedIntText(value=3, description='Image Num:', max=20)

Button(description='Generate', style=ButtonStyle())

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]